In [1]:
import main

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
import pickle

In [3]:
random_board_pairs = []
with open("2021-08-11-3ply-v2-boards6-random.pickle", "rb") as f:
    while True:
        random_board_pairs.append(pickle.load(f))

EOFError: Ran out of input

In [4]:
len(random_board_pairs)

24483

In [5]:
game_board_pairs = []
with open("2021-08-16-3ply-v2-boards7-games.pickle", "rb") as f:
    while True:
        game_board_pairs.append(pickle.load(f))

EOFError: Ran out of input

In [6]:
len(game_board_pairs)

28230

In [7]:
random_inputs = []
random_outputs = []
for p in random_board_pairs:
    random_inputs.append(p[0].get_flipped_3d_array())
    random_outputs.append(float(p[1]))

In [8]:
game_inputs = []
game_outputs = []
for p in game_board_pairs:
    game_inputs.append(p[0].get_flipped_3d_array())
    game_outputs.append(float(p[1]))

In [9]:
import numpy as np

In [10]:
game_inputs = np.array(game_inputs)

In [11]:
random_inputs = np.array(random_inputs)

In [12]:
game_outputs = np.array(game_outputs)

In [13]:
random_outputs = np.array(random_outputs)

In [14]:
game_outputs.shape

(28230,)

In [15]:
game_outputs[0]

0.5675877332687378

In [16]:
random_mark = int(len(random_outputs)*0.8)

In [17]:
game_mark = int(len(game_outputs)*0.8)

In [18]:
game_mark

22584

In [19]:
random_mark

19586

In [20]:
train_inputs = np.concatenate((random_inputs[0:random_mark] , game_inputs[0:game_mark] 
                               , random_inputs[random_mark:] ,  game_inputs[game_mark:])) 

In [21]:
train_outputs = np.concatenate(( random_outputs[0:random_mark],
                                 game_outputs[0:game_mark],
                                random_outputs[random_mark:] ,
                                game_outputs[game_mark:] ))

In [22]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [23]:
model2 = keras.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [24]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
Total params: 19,104
Trainable params: 19,104
Non-trainable params: 0
_________________________________________________________________


In [25]:
model2.add(layers.Flatten())
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(1, activation='relu'))

In [26]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 10560)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               2703616   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,722,977
Trainable params: 2,722,977
Non-trainable params: 0
_________________________________________________________________


In [27]:
model2.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [28]:
model2.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1318/1318 [==============================] - 29s 22ms/step - loss: 0.0986 - val_loss: 0.0406
Epoch 2/10
1318/1318 [==============================] - 29s 22ms/step - loss: 0.0365 - val_loss: 0.0363
Epoch 3/10
1318/1318 [==============================] - 31s 23ms/step - loss: 0.0319 - val_loss: 0.0332
Epoch 4/10
1318/1318 [==============================] - 48s 36ms/step - loss: 0.0307 - val_loss: 0.0324
Epoch 5/10
1318/1318 [==============================] - 36s 27ms/step - loss: 0.0294 - val_loss: 0.0308
Epoch 6/10
1318/1318 [==============================] - 35s 27ms/step - loss: 0.0271 - val_loss: 0.0298
Epoch 7/10
1318/1318 [==============================] - 41s 31ms/step - loss: 0.0261 - val_loss: 0.0298
Epoch 8/10
1318/1318 [==============================] - 34s 26ms/step - loss: 0.0249 - val_loss: 0.0309
Epoch 9/10
1318/1318 [==============================] - 31s 24ms/step - loss: 0.0237 - val_loss: 0.0303
Epoch 10/10
1318/1318 [==============================] - 37s 28m

In [29]:
model2.save('saved_model/2021-08-23-v3-model')

INFO:tensorflow:Assets written to: saved_model/2021-08-23-v3-model/assets
